<a href="https://colab.research.google.com/github/patgarmarq/03MAIR---Algoritmos-de-Optimizacion---2019/blob/master/Patricia_Garcia_AG3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Actividad Guiada 3 <br>
Nombre: Patricia García Marquina. <br>
Link:https://colab.research.google.com/drive/1lPHliV73918eJiuDv4tTOd4P9NtClyyN<br>
Git (en Master):https://github.com/patgarmarq/03MAIR---Algoritmos-de-Optimizacion---2019 <br>



# Notas al profesor:
- Los apartados de este archivo contienen:

  - Problema 1.Agente viajero - TSP. TSPLIB
  - Caso 1: Búsqueda aleatoria
  - Caso 2: Búsqueda local. Generador de vecindad
  - Caso 3: Recocido simulado
  - caso 4: Método constructivo. Colonia de Hormigas


# __Programación dinámica__

## __Problema 1__. Agente viajero

- Datos que estudiar: http://elib.zib.de/pub/mp-testdata/tsp/tsplib/tsp/swiss42.tsp

  - Matriz completa que representa de 42 ciudades. Siendo Origen la vertical y Destino la horizontal.

- Asumimos que todos los nodos están conectados y comenzamos y terminamos por el nodo 0

In [8]:
#Instalacion y descarga del fichero
!pip install requests # Instalar la librería

import urllib.request

file = "swiss42.tsp"
urllib.request.urlretrieve("http://elib.zib.de/pub/mp-testdata/tsp/tsplib/tsp/swiss42.tsp", file)

('swiss42.tsp', <http.client.HTTPMessage at 0x7fe8383fa9b0>)

In [10]:
# Instalar la libreria para problemas de TSP (agente viajero)
!pip install tsplib95  

In [0]:
import tsplib95
import random
import math
problem = tsplib95.load_problem(file) 
#Creamos lista de nodos
Nodos = list(problem.get_nodes()) 
#Aristas
Aristas = list(problem.get_edges())    


# __Definición de funciones__

- Vamos a definir tres funciones 
  - __crear_solucion__: Recibe la lista de nodos y devuelve una solución aleatoria.
  - __distancia__ devuelve la distancia entre dos nodos
  - __distancia_total__ que se utilizará
  
- Metaheuristicas hay que repetir y buscar la siguiente funcion

In [12]:
#Se genera una solucion aleatoria con comienzo en en el nodo 0
def crear_solucion(Nodos): 
  solucion = [0]
  for i in range(len(Nodos)-1):
    #Toma un nodo no elegido aleatoriamente. Set no importa el orden
    solucion = solucion + [random.choice(list(set(Nodos) - set(solucion)))] 
  return solucion  

#Devuelve la distancia entre dos nodos
def distancia(a,b, problem):
  return problem.wfunc(a,b)

#Devuelve la distancia total de una trayectoria
def distancia_total(solucion, problem):
  distancia_total = 0
  for i in range(len(solucion)-1):
    distancia_total += distancia(solucion[i] ,solucion[i+1] ,  problem)
  return distancia_total + distancia(solucion[len(solucion)-1] ,solucion[0], problem)

print("Distancia 0-1: ",distancia(0,1,problem))  #Dist nodo 0 y el 1, de problem (donde tengo mis datos)

solucion = crear_solucion(Nodos)
print("Distancia total ",distancia_total(solucion,problem))

Distancia 0-1:  15
Distancia total  4878


# **Caso 1: Búsqueda aleatoria**: 

In [13]:
#Busqueda aleatorio. N: nº iteraciones
def busqueda_aleatoria(problem, N):
  mejor_solucion = crear_solucion(Nodos)
  mejor_valor    = distancia_total(mejor_solucion, problem) # dis(solucion)

  for i in range(N):     # iteraciones. Va generando nuevas iteraciones
      solucion = crear_solucion(Nodos)
      valor    = distancia_total(solucion, problem)
      if valor < mejor_valor:
        mejor_solucion = solucion
        mejor_valor = valor

  print("La mejor solucion es ",mejor_valor,  mejor_solucion)
   
#busqueda_aleatoria(problem, 20000)
print("Búsqueda aleatoria:\n")
sol_busq_aleatoria = busqueda_aleatoria(problem, 100)

Búsqueda aleatoria:

La mejor solucion es  3986 [0, 15, 13, 2, 17, 27, 24, 29, 8, 18, 10, 39, 7, 21, 6, 32, 3, 4, 30, 9, 22, 34, 20, 35, 36, 16, 19, 37, 26, 40, 41, 11, 33, 12, 14, 28, 23, 25, 38, 31, 1, 5]


In [0]:
busqueda_aleatoria(problem, 20000)

La mejor solucion es  3427 [0, 13, 12, 18, 11, 14, 10, 8, 22, 21, 39, 34, 20, 29, 26, 30, 40, 24, 32, 41, 27, 17, 36, 1, 31, 35, 4, 2, 7, 37, 15, 5, 6, 25, 9, 23, 16, 19, 28, 3, 38, 33]


# **Caso 2: Búsqueda local**: 

__Generador de vecindad__

- Se genera una lista que permita encontrar la mejor solución vecina a una solución (aleatoria) dada.

- Vamos a utilizar el operador 2-opt (que es intercambiar 2 nodos)
  - Para el problema de 42 ciudades debo intercambiar 41*42 =820 posibilidades (y /2 porque elimino las soluciones simétricas)

In [0]:
#La funcion recibe una solucion cualquiera y usamos intercambiador de nodos

def genera_vecina(solucion):
  #Generador de soluciones vecinas: 2-opt (intercambiar 2 nodos) Si hay N nodos se generan (N-1)x(N-2)/2 soluciones
  #print(solucion)
  mejor_solucion = []
  mejor_distancia = 10e100


  for i in range(1,len(solucion)-1):
    for j in range(i+1, len(solucion)):

      #Me quedo con la 1º parte de la solución (desde 0 a i) y luego intercambio j por i, 
      # pego entre medias lo que hay entre i+1 y j
      vecina = solucion[:i] + [solucion[j]] + solucion[i+1:j] + [solucion[i]] + solucion[j+1:] 
      distancia_vecina = distancia_total(vecina, problem)
      if distancia_vecina <= mejor_distancia:
        mejor_distancia = distancia_vecina
        mejor_solucion = vecina
  return mejor_solucion
      

solucion = crear_solucion(Nodos)
mejor_solucion = genera_vecina(solucion)




In [16]:
###################################
def busqueda_local(problem, N):
  #problem = datos del problema
  #N = numero de iteraciones
  
  mejor_solucion = []
  mejor_distancia = 10e100
  
  solucion_referencia = crear_solucion(Nodos)   # Referencia solo
  
  #Creamos vecinos y comparamos distancias
  for i in range(N):
    vecina = genera_vecina(solucion_referencia)    
    distancia_vecina = distancia_total(vecina, problem)
    if distancia_vecina <= mejor_distancia:
      mejor_solucion = vecina
      mejor_distancia = distancia_vecina
        
    solucion_referencia = vecina #va dentro del if o fuera? el lo pone fuera, pero mejores res. dentro
  print("La mejor solución encontrada es " , mejor_distancia , mejor_solucion)

print("\nBusqueda local: Este algoritmo tarda algo más, pero encuentra mejores soluciones, (record 1273)\n")

sol_bus_local = busqueda_local(problem, 500)#Por búsqueda local, cada vez estamos mejorando la búsqueda local. Tenemos mejores soluciones
#y proximas a la mejor solución encontrada aprox  record 1273


Busqueda local: Este algoritmo tarda algo más, pero encuentra mejores soluciones, (record 1273)

La mejor solución encontrada es  1854 [0, 4, 10, 41, 23, 9, 22, 38, 7, 14, 16, 19, 13, 11, 25, 12, 5, 15, 37, 17, 36, 35, 31, 1, 6, 26, 18, 8, 40, 24, 21, 39, 29, 30, 28, 32, 20, 33, 34, 27, 2, 3]


Tenderá a minimos locales, llegará un punto que llegue a un mínimo local y por ello __debemos buscar estrategias__ para salir del mínimo local:
  - Modificar estructuras de entornos  búsqueda en _entornos variables.
  - Permitir movimientos peores respecto a la solución actual, _búsqueda tabú_, _recocido simulado_.
  - Volver a comenzar con otras soluciones iniciales: búsquedas _multiarranque_

# __caso 3: Recocido simulado__
__Generar una solución vecina de una solución dada__
  - elegir dos nodos al azar (los ordenamos), y los intercambiamos
  - Nos da casi la misma solución, pero con algún intercambio de nodos



In [21]:
def genera_vecina_aleatorio(solucion):
  #Generador de 1 solucion vecina 2-opt (intercambiar 2 nodos)
  #Se puede mejorar haciendo que la elección no se uniforme sino entre las que estén más proximas
  i = random.choice(range(1, len(solucion)) )
  j = random.choice(list(set(range(1, len(solucion))) - {i}))
  if j<i:
    i,j=j,i
  vecina = list(solucion[:i] + [solucion[j]] + solucion[i+1:j] + [solucion[i]] + solucion[j+1:])
  return vecina

solucion = crear_solucion(Nodos)
sol_vecina_aleatorio = genera_vecina_aleatorio(solucion)
print("Solución\n",solucion,"\nSolución genera_vecina_aleatorio\n", sol_vecina_aleatorio)
print("\n Doble intercambio\n", genera_vecina_aleatorio(genera_vecina_aleatorio(solucion)))
#Es casi la misma, pero con dos elementos de la lista anterior y se intercambian. 

Solución
 [0, 41, 14, 33, 2, 19, 22, 39, 20, 9, 24, 21, 29, 25, 36, 18, 15, 37, 28, 32, 5, 10, 16, 31, 4, 7, 8, 27, 23, 35, 30, 34, 13, 38, 3, 12, 11, 1, 17, 26, 6, 40] 
Solución genera_vecina_aleatorio
 [0, 41, 21, 33, 2, 19, 22, 39, 20, 9, 24, 14, 29, 25, 36, 18, 15, 37, 28, 32, 5, 10, 16, 31, 4, 7, 8, 27, 23, 35, 30, 34, 13, 38, 3, 12, 11, 1, 17, 26, 6, 40]

 Doble intercambio
 [0, 41, 14, 33, 2, 5, 22, 39, 20, 9, 24, 21, 29, 28, 36, 18, 15, 37, 25, 32, 19, 10, 16, 31, 4, 7, 8, 27, 23, 35, 30, 34, 13, 38, 3, 12, 11, 1, 17, 26, 6, 40]


#### __Funciones para Probabilidad, Temperatura__
  - Cálculo de la probabilidad. ¿cuando nos quedamos con una solución peor? Se aplica la fórmula $P_{aceptacion} = e^{\frac{-\delta}{T}}$
    - delta ($\delta$) : la diferencia entre solución actual y de referencia (vecina).
    - Temperatura ($T$): la temperatura actual.
  
  - __Función descenso de temperatura__ En este caso el exponencial, desciende la T en funcón de un parámetro fijo, aunque podríamos aplicar los siguiente:
    - __Descenso constante__
    - __Basado en descensos sucesivos__ por tramos dependiendo de la iteración.
    - __Descenso exponencial (usada aqui) o descenso de gradiente__: $T_{k+1} = \alpha · T_K$
    - __Criterio Boltzmann:__ $T_k = \frac{T_0}{1 + log(k)}$
    - __Esquema de Cauchy__: $T_k = \frac{T_0}{1+k}$



    

In [0]:
#Funcion de probabilidad para determinar si se cambia 
#    a una solución peor respecto a la de referencia(exponencial)
def probabilidad(T,d):
  if random.random() <= math.exp(-1*d / T)   :
    return True
  else:
    return False


def bajar_temperatura(T):
  return T*.9999

## __Proceso principal Recocido simulado__
- Mientras T >1:
  - Genero solucion vecina aleatoria (nodos al azar e intercambiarlos)
  - Calcula el valor de la nueva sol vecina.
  - Si esa dist es mejor que la que teníamos antes -> la guardamos

  - Ademas, calculamos __probabilidad__ de las funciones anteriores, en función de la temperatura y la distancia entre la nueva distancia y la que teníamos de referencia.
  - __Si mejora la distancia__ me quedo con ella.
  - __SI NO MEJORA LA DISTANCIA__ obtenemos una probabilidad de poder quedármela aunque sea peor (así obtenemos los casos peores para este algoritmo).

- Cuando se llega a la Temperatura = 1, saca la mejor solución encontrada

In [24]:
#####################################################################
def recocido_simulado(problem, TEMPERATURA=1000):
  #problem = datos del problema

  solucion_referencia = crear_solucion(Nodos)
  distancia_referencia = distancia_total(solucion_referencia, problem)
  
  mejor_solucion = []
  mejor_distancia = 10e100

  while TEMPERATURA > 1:
    #Genera una solución vecina(aleatoria)
    vecina = genera_vecina_aleatorio(solucion_referencia) 
    
    #Calcula su valor(distancia) la solucion 
    distancia_vecina = distancia_total(vecina, problem)
    
    #Si es la mejor solución de todas se guarda
    if distancia_vecina < mejor_distancia:
      mejor_solucion = vecina
      mejor_distancia = distancia_vecina
      print('\t****Encontrada mejor solucion:', mejor_distancia , mejor_solucion, '\n')
    
    #Si la nueva vecina es mejor se cambia y si es peor se cambia según 
    # una probabilidad dependiente de T y de |distancia_referencia - distancia_vecina|
    p = probabilidad(TEMPERATURA, abs(distancia_referencia - distancia_vecina) )
    if distancia_vecina < distancia_referencia or p : # Si no es mejor, veo p
      solucion_referencia = vecina
      distancia_referencia = distancia_vecina
    
    TEMPERATURA = bajar_temperatura(TEMPERATURA)

  print("La mejor solución encontrada es :" , mejor_distancia, mejor_solucion  )
  return mejor_solucion

sol_recocido_simulado = recocido_simulado(problem, TEMPERATURA=100000) #Solo imprime si encuentra mejor sol/iteracion


	****Encontrada mejor solucion: 5170 [0, 3, 33, 30, 25, 27, 10, 23, 35, 24, 8, 28, 9, 16, 38, 19, 22, 21, 37, 11, 31, 34, 15, 29, 7, 14, 41, 26, 1, 4, 36, 39, 32, 13, 18, 6, 12, 2, 5, 17, 20, 40] 

	****Encontrada mejor solucion: 5160 [0, 3, 33, 30, 25, 27, 10, 23, 35, 24, 8, 28, 9, 16, 38, 19, 22, 21, 37, 11, 31, 34, 15, 29, 7, 14, 41, 26, 1, 4, 36, 39, 32, 18, 13, 6, 12, 2, 5, 17, 20, 40] 

	****Encontrada mejor solucion: 5026 [0, 3, 33, 30, 25, 39, 10, 23, 35, 24, 8, 28, 9, 16, 38, 19, 22, 21, 37, 11, 31, 34, 15, 29, 7, 14, 41, 26, 1, 4, 36, 27, 32, 18, 13, 6, 12, 2, 5, 17, 20, 40] 

	****Encontrada mejor solucion: 4999 [0, 3, 33, 30, 25, 39, 10, 23, 35, 24, 8, 28, 9, 16, 38, 17, 22, 21, 37, 11, 31, 34, 15, 29, 7, 14, 41, 26, 1, 4, 36, 27, 32, 18, 13, 6, 12, 2, 5, 19, 20, 40] 

	****Encontrada mejor solucion: 4976 [0, 3, 33, 30, 25, 39, 10, 23, 35, 24, 8, 28, 41, 16, 38, 17, 22, 21, 37, 11, 31, 34, 15, 29, 7, 14, 9, 26, 1, 4, 36, 27, 32, 18, 2, 6, 12, 13, 5, 19, 20, 40] 

	****Encon

# __caso 4: Método constructivo. Colonia de Hormigas__
- ¿Que elegimos de la siguiente ciudad?
  - Distancias con el nodo de referencia.
  - Las distancias con más feromonas.
  - Se incluye un componente aleatorio, para ser capaces de explorar nuevos caminos, que no se "están explorando" normalmente por las hormigas, pero que "de vez en cuando" se va teniendo en cuenta por los siguientes "agentes u hormigas"


__Conclusiones__: No baja le valor de la solución porque estamso usando operadores muy simples, y tenemos un componente aleatorio MUY elevado por la función nodo

In [26]:
# Funciones auxiliares. Colonia de hormigas

#Poco eficiente, demasiado aleatoria. H solucion
def Add_Nodo(problem, H ,T ) :
  #Establecer una una funcion de probabilidad para añadir un nuevo nodo 
  # dependiendo de los nodos mas cercanos y de las feromonas depositadas
  Nodos = list(problem.get_nodes())
  return random.choice(   list(set(range(1,len(Nodos))) - set(H) )  ) 
  # Elige un nodo cualquiera de los que "no han sido elegidos"

#Refuerza las soluciones que sean buenas
def Incrementa_Feromona(problem, T, H):
  #Incrementar segun la calidad de la solución. Añadir una cantidad 
  # inversamente proporcional a la distancia total 
  for i in range(len(H)-1):
    T[H[i]][H[i+1]] += 1000/distancia_total(H, problem)
  return T

def Evaporar_Feromonas(T):
  #Podemos elegir diferentes funciones de evaporación dependiendo de la 
  # cantidad actual y de la suma total de feromonas depositadas,...
  #Evapora 0.3 el valor de la feromona, sin que baje de 1
  T = [[ max(T[i][j] - 0.3 , 1) for i in range(len(Nodos)) ] for j in range(len(Nodos))]
  return T
#_______________________________________________________________________________  
# PRIMERA ITERACION.
# Problem : datos del problema
# N = Número de agnetes (hormigas)

def hormigas(problem, N) :
  #problem = datos del problema
  #N = Número de agentes(hormigas)
    
  #Nodos
  Nodos = list(problem.get_nodes())
  #Aristas
  Aristas = list(problem.get_edges()) 
  
  #Inicializa las aristas con una cantidad inicial de feromonas:1
  T = [[ 1 for _ in range(len(Nodos)) ] for _ in range(len(Nodos))]
  
  #Se generan los agentes(hormigas) que serán estructuras de caminos desde 0
  Hormiga = [[0] for _ in range(N)]  #inicializo todo a ceros

  #_____________________________________________________________________________
  #Siguientes iteraciones
  #Recorre cada agente construyendo la solución (h simula el número de hormiga )
  for h in range(N) :
    #print("\nAgente:", h)
    #Para cada agente se construye un camino
    for i in range(len(Nodos)-1) :
      
      #Elige el siguiente nodo
      Nuevo_Nodo = Add_Nodo(problem, Hormiga[h] ,T )
      Hormiga[h].append(Nuevo_Nodo)     
    
    #Incrementa feromonas en esa arista 
    T = Incrementa_Feromona(problem, T, Hormiga[h] )
    #print("Feromonas(1)", T)
      
    #Evapora Feromonas  
    T = Evaporar_Feromonas(T)
    #print("Feromonas(2)", T)

    #Seleccionamos el mejor agente
  mejor_solucion = []
  mejor_distancia = 10e100
  for h in range(N) :
    distancia_actual = distancia_total(Hormiga[h], problem)
    if distancia_actual < mejor_distancia:
      mejor_solucion = Hormiga[h]
      mejor_distancia =distancia_actual
  
 
  print("Mejor solucion", mejor_solucion,"\n")
  print("\nMejor distancia:\n",mejor_distancia)

# No baja le valor de la solución porque estamso usando operadores muy simples,
# y enemos un componente aleatorio MUY elevado por la función nodo
hormigas(problem,1000)

Mejor solucion [0, 8, 21, 12, 26, 3, 27, 6, 19, 35, 30, 38, 24, 2, 9, 29, 10, 18, 4, 16, 20, 25, 5, 41, 11, 40, 39, 32, 1, 33, 13, 15, 36, 37, 14, 34, 23, 22, 31, 7, 17, 28] 


Mejor distancia:
 3940
